In [1]:
!pip install "sap-llm-commons[all]"

In [2]:
!pip install ai-api-client-sdk

In [1]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

C:\Users\kylin\AppData\Local\Temp\ipykernel_3840\3646097473.py:5: UserWarning: Starting from verison 1.0.0 the default proxy_version was set to 'gen-ai-hub'. Use gen_ai_hub.proxy.core.proxy_clients.set_proxy_version('btp') to set the proxy_version to 'btp'.
  import llm_commons.proxy.base


In [2]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [3]:
resource_group = widgets.Text(
    value='default', # resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False
)

resource_group

Text(value='default', placeholder='Resource group of deployments')

In [4]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

In [5]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'

llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']

In [6]:
aic_proxy_client = AICoreProxyClient()
btp_proxy_client = BTPProxyClient()

Deploy customized fine-tuned models

In [7]:
AICoreProxyClient.add_foundation_model_scenario(
    scenario_id='fine-tuned-llm',
    config_names='fine-tuned-*',
    prediction_url_suffix='/v1/completions'
)

In [8]:
aic_proxy_client.get_deployments()

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc8e3c266c8881f0', config_id='58a5ffea-1afe-44b4-b6b9-ca749776e58e', config_name='tiiuae--falcon-40b-instruct-config', deployment_id='dc8e3c266c8881f0', model_name='tiiuae--falcon-40b-instruct', created_at=datetime.datetime(2024, 1, 25, 12, 53, 26, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'aicore-opensource', 'model_version': 'null'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dd88f66091ca1982', config_id='a99e09fb-7e88-4a8d-abe8-af7f8b146cb8', config_name='text-embedding-ada-002-config', deployment_id='dd88f66091ca1982', model_name='text-embedding-ada-002', created_at=datetime.datetime(2024, 1, 25, 12, 16, 20, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '2'}, custom_prediction_suffix=None),
 Deployment(url='https

In [11]:
#text_davinci_003 = Completion(deployment_id='text-davinci-003')

In [15]:
#text_davinci_003.create(model_name='our-awesome-model/v1', prompt="San Francisco is a")

ValueError: 'deployment_id' and 'url' have to be of type str

Initialize harmonized model

In [9]:
llm = init_llm('gpt-35-turbo', proxy_client = aic_proxy_client,temperature=0., max_tokens=256, deployment_id='d10917558b77a3db', api_base=llm_commons.proxy.api_base)

In [10]:
llm.predict("发财树叶子发黄是为什么")

j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'发财树叶子发黄可能是由于以下原因：\n\n1. 过浇水：发财树喜欢湿润但排水良好的土壤，如果土壤过于湿润，根部可能会受到损害，导致叶子发黄。\n\n2. 缺水：发财树叶子发黄也可能是因为缺水，如果土壤过于干燥，植物无法吸收足够的水分和养分，导致叶子变黄。\n\n3. 光照不足：发财树需要充足的阳光才能进行光合作用，如果光照不足，叶子可能会变黄。\n\n4. 营养不良：如果土壤中缺乏必要的养分，如氮、磷、钾等，发财树的叶子也会'

In [13]:
embedding = init_embedding_model('text-embedding-ada-002', proxy_client=aic_proxy_client, deployment_id='dd88f66091ca1982', api_base=llm_commons.proxy.api_base)

KeyError: "Model 'text-embedding-ada-002-config' is not available!"

In [28]:
query = "What is the world's best B2B company?"
query_embedding = embedding.embed_query(query)

In [21]:
documents = ["This is a sample document"]
document_embedding = embedding.embed_documents(documents)

AI Core proxy support

In [30]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
chat = ChatOpenAI(proxy_model_name='gpt-4')
chat.predict("What is the world's best B2B company?")

'There are many top-tier B2B companies, and the "best" can vary depending on specific criteria such as revenue, customer satisfaction, or innovation. However, some of the world\'s leading B2B companies include:\n\n1. IBM: A multinational technology company that provides hardware, software, cloud-based services, and cognitive computing.\n\n2. Microsoft: They offer a wide range of software products, cloud services, and hardware. Their B2B solutions include Azure, Dynamics 365, and Office 365.\n\n3. Amazon Web Services (AWS): Amazon\'s cloud computing platform provides a wide range of services to businesses across the globe.\n\n4. Salesforce: This company is a global leader in customer relationship management (CRM) software.\n\n5. Siemens: A global conglomerate that offers a variety of services and products, including industrial automation, energy, and healthcare solutions.\n\n6. General Electric (GE): They offer products and services in sectors like aviation, power, renewable energy, dig

Proxy client - use proxies in parallel

In [23]:
from llm_commons.langchain.proxy import ChatOpenAI
from langchain.schema.messages import HumanMessage
from llm_commons.proxy.openai import ChatCompletion

In [24]:
messages=[{"role": "user", "content": "Say this is a test"}]

print(ChatCompletion.create(proxy_client=aic_proxy_client, model_name='gpt-4', messages=messages, api_base=os.environ['AICORE_LLM_API_BASE']))

{
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "This is a test.",
        "role": "assistant"
      }
    }
  ],
  "created": 1706845902,
  "id": "chatcmpl-8nesYYfWriCKl5ngRAok9SOwLjvsh",
  "model": "gpt-4",
  "object": "chat.completion",
  "prompt_filter_results": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "jailbreak": {
          "detected": false,
          "filtered": false
        },
        "self_harm": {
          "filtered": false

In [25]:
messages = [
    HumanMessage(content="Say this is a test"),
]
# btp_llm = ChatOpenAI(proxy_client=btp_proxy_client, deployment_id='gpt-4')
# print(btp_llm.invoke(messages))
aic_llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
print(aic_llm.invoke(messages))

content='This is a test.'


Using llm-commons with fine-tuned models

In [26]:
# use it with langchain

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from llm_commons.langchain.proxy import OpenAI

llm = ChatOpenAI(proxy_model_name='gpt-4')
template = "Question: {question}\nAnswer: Let's think step by step."""

In [27]:
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.run("What is the world's best B2B company?")



> Entering new LLMChain chain...
Prompt after formatting:
Question: What is the world's best B2B company?
Answer: Let's think step by step.

> Finished chain.


'There isn\'t a definitive answer to this question, as the "best" B2B company can vary depending on specific criteria such as industry, scale, innovation, customer service, etc. Some of the world\'s top B2B companies include Microsoft, Salesforce, and Alibaba. However, it is important to consider the specific needs and objectives of your own company when deciding which B2B company is the "best" for you to partner with.'